# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-wit

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/
https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineerin

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/mistralai/Devstral-Small-2505',
 '/ByteDance-Seed/BAGEL-7B-MoT',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/google/medgemma-4b-it',
 '/nari-labs/Dia-1.6B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Lightricks/ltx-video-distilled',
 '/spaces/NihalGazi/FLUX-Pro-Unlimited',
 '/spaces/stepfun-ai/Step1X-3D',
 '/spaces/ByteDance/DreamO',
 '/spaces',
 '/datasets/openbmb/Ultra-FineWeb',
 '/datasets/disco-eth/EuroSpeech',
 '/datasets/ministere-culture/comparia-conversations',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/tr

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
mistralai/Devstral-Small-2505
Updated
2 days ago
•
45.9k
•
507
ByteDance-Seed/BAGEL-7B-MoT
Updated
3 days ago
•
998
•
484
google/gemma-3n-E4B-it-litert-preview
Updated
5 days ago
•
457
google/medgemma-4b-it
Updated
4 days ago
•
8.49k
•
18

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [26]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [23]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmistralai/Devstral-Small-2505\nUpdated\n2 days ago\n•\n64.4k\n•\n514\nByteDance-Seed/BAGEL-7B-MoT\nUpdated\n3 days ago\n•\n1.5k\n•\n513\ngoogle/gemma-3n-E4B-it-litert-preview\nUpdated\n5 days ago\n•\n464\ngoogle/medgemma-4b-it\nUpdated\n4 days ago\n•\n10.1k\n•\n195\nnari-labs/Dia-1.6B\nUpdated\n11 days ago\n•\n168k\n•\n2.39k\nBrowse 1M+ models\nSpaces\nRunning\n7.14k\n7.14k\nDeepSite\n🐳\nG

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [24]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Company Brochure

## About Us
**Hugging Face** is dedicated to building a collaborative AI community that shapes the future of machine learning. With a platform that facilitates the sharing of models, datasets, and AI applications, Hugging Face empowers thousands of organizations worldwide—ranging from startups to large enterprises—to leverage cutting-edge machine learning resources.

### Our Mission
We aim to democratize AI through open-source collaboration and knowledge sharing, enabling everyone from researchers to developers to contribute and benefit from the advancements in machine learning.

## Company Culture
At Hugging Face, our culture revolves around collaboration, innovation, and inclusivity. We encourage our team members to engage openly with the community and embrace the spirit of sharing insights and tools. We celebrate the contributions of individuals and acknowledge the importance of diverse voices in shaping the future of AI.

### Community Engagement
- **Open Source Philosophy**: We believe in the power of community-driven development and provide a platform that supports everything from models to datasets collaboratively contributed by users.
- **Events and Workshops**: Regular community events, discussions, and releases to engage and educate both novice and seasoned developers in the AI field.

## Products and Services
Hugging Face provides a variety of offerings to meet the needs of its users:

### 1. **Models**: 
- Access to over 1 million models and 250,000 datasets including various state-of-the-art architectures for text, image, audio, and video tasks.
  
### 2. **Spaces**: 
- An application hosting service that allows developers to create and share AI applications seamlessly.
  
### 3. **Enterprise Solutions**: 
- Tailored services for organizations looking for advanced security, compliance, and dedicated support. These solutions include Single Sign-On, resource management, and priority support.

### 4. **Community and Documentation**: 
- Rich documentation and resources available for users to start building and deploying their ML applications effectively.

## Customers
More than 50,000 organizations are leveraging Hugging Face’s platform, including industry giants like:
- Google
- Microsoft
- Amazon
- Meta
- Grammarly

Our solutions cater to a diverse clientele from educational institutions and non-profits to Fortune 500 companies, illustrating the versatility and adaptability of our products.

## Career Opportunities
Join us in shaping the future of artificial intelligence! We are always looking for passionate individuals who thrive in a collaborative environment. Available positions range from software engineers to data scientists and community managers.

### Why Work at Hugging Face?
- **Innovative Projects**: Participate in exciting AI projects that directly impact the technology landscape.
- **Remote-Friendly**: Work from anywhere while being part of an inclusive team.
- **Competitive Benefits**: We offer attractive compensation packages, professional development opportunities, and a supportive work environment.

For a list of current job openings, visit our [careers page](https://huggingface.co/join). 

## Get In Touch
For more information about Hugging Face or to explore partnership opportunities, please visit our website [Hugging Face](https://huggingface.co) or engage with our community on social media:

- **GitHub**
- **Twitter**
- **LinkedIn**
- **Discord**

Join us as we build a future powered by AI, one model at a time!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [25]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face: Your Partner in AI Development

## About Us
Hugging Face is at the forefront of the AI revolution, embodying the spirit of community collaboration to build the future. Our platform empowers researchers and practitioners in the machine learning field to collaborate on models, datasets, and applications efficiently. With over 1 million models available and numerous datasets, Hugging Face serves as the ultimate space for innovation and discovery in AI.

## Our Services
- **Models:** Explore and utilize over 1 million machine learning models ranging from text, image, video, audio, to 3D modalities.
- **Datasets:** Access and share over 250,000 datasets for any machine learning task.
- **Spaces:** Create and host applications effortlessly on Hugging Face Spaces—your canvas for showcasing machine learning applications.
- **Enterprise Solutions:** Tailored enterprise-grade solutions with enhanced security, support, and compute capabilities. Starting at $20/user/month.

## Who We Serve
With over 50,000 organizations utilizing our platform, we cater to a diverse clientele, from startups to tech giants such as Google, Amazon, Microsoft, and NVIDIA. Our community-driven approach ensures that everyone—from researchers to large enterprises—can harness the power of AI effectively.

## Company Culture
At Hugging Face, we are driven by our mission to democratize machine learning. We believe in open communication and collaboration, ensuring that every team member's voice is heard. Our work environment fosters creativity and innovation, encouraging continuous learning and development.

### Join Our Community
We invite passionate individuals to become part of our diverse workforce. Whether you're looking to contribute to open-source projects, develop AI technologies, or support our community-centric mission, there is a place for you at Hugging Face. 

## Career Opportunities
Hugging Face is always on the lookout for talented individuals to join us on our journey. We offer competitive salaries, opportunities for growth, and a supportive workplace for development and collaboration in AI. Check our [careers page](https://huggingface.co) for current openings.

## Get in Touch
Interested in what we do? Explore our offerings or join the AI community at Hugging Face:
- [Visit Our Website](https://huggingface.co)
- Follow us on [Twitter](https://twitter.com/huggingface) | [LinkedIn](https://linkedin.com/company/huggingface) | [GitHub](https://github.com/huggingface) | [Discord](https://discord.gg/huggingface)

Let's build the future of AI together.

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face Brochure

## The AI Community Building the Future

At Hugging Face, we are at the forefront of the AI revolution, offering a collaborative platform where the machine learning community comes together to build models, share datasets, and develop applications. Our aim is to accelerate your machine learning journey and make groundbreaking AI accessible to everyone.

### Key Offerings

- **Explore AI Models**: Dive into a collection of over 1 million models, including the latest advancements and trending technologies in AI. Popular models include:
  - **mistralai/Devstral-Small-2505**: 45.9k followers
  - **ByteDance-Seed/BAGEL-7B-MoT**: 998 followers

- **Datasets**: Access a vast repository of over 250k datasets, designed for various machine learning tasks.

- **Spaces**: Run and share unlimited applications on our platform. Currently, we host over 400k applications that cater to diverse use cases.

- **Enterprise Solutions**: For organizations looking to integrate AI at scale, we offer enterprise-grade solutions with enhanced security, support, and dedicated resources starting at $20/user/month.

### Our Community

With over 50,000 organizations using our platform, including notable companies like Amazon, Google, and Microsoft, we pride ourselves on fostering a vibrant community. Hugging Face serves as a communal hub for collaboration, encouraging innovation in every aspect of machine learning.

### Open Source Commitment

At the heart of Hugging Face is our commitment to open-source development. Our frameworks, including:
- **Transformers**: For state-of-the-art machine learning in PyTorch, TensorFlow, and JAX
- **Diffusers**: Leading diffusion models
- **Tokenizers**: Fast tokenizers optimized for both research and production

We actively engage with the community to continually improve our tooling and resources.

### Company Culture

We believe in nurturing a collaborative, inclusive, and innovative workplace. Our culture emphasizes:
- **Community**: Working together to achieve common goals.
- **Transparency**: Open dialogue and shared knowledge.
- **Empowerment**: Encouraging continuous learning and growth.

### Careers at Hugging Face

Join us in shaping the future of AI! We are looking for passionate individuals across various domains to help us push the boundaries of machine learning. Visit our [jobs page](#) to discover current openings.

---

For more information, visit our website at [Hugging Face](https://huggingface.co) or follow us on LinkedIn, Twitter, and Discord to stay updated on our latest developments.



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>